# 공공데이터크롤링/오픈API를 이용한 한국에 입국한 일본인 수 크롤링 프로젝프

## 1) 공공데이터 활용신청

- https://www.data.go.kr/index.do에 접속하여 '출입국관광통게서비스'를 검색한다.
- [오픈 API] 탭을 선택한 뒤 API 목록에서 [출입국관광통계서비스]를 클릭 후 OpenAPI 개발계정을 발급받는다.

## 2) 함수 설계하기

In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

ServiceKey = "iFTTlGgsdOVZ316AlUObz4TyTtzaPPemlGB02QRUII5cjHVpq%2FZaVAaQFnGVEn%2FYY8Uk44JjELB2IV0vzbDr%2FA%3D%3D"

def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

- 12행 매개변수로 받은 url에 대한 요청을 보낼 객체를 생성
- 14행 요청 객체를 보내서 받은 응답 데이터를 response 객체에 저장
- 15~17행 response 객체에 저장된 코드를 확인 코드가 200이면 요청을 정상 처리한 것이므로 성공메시지와 현재 시간을 파이썬 셸 창에 출력하고 응답을 utf-8 형식으로 디코딩하여 반환
- 18~21행 요청이 처리되지 않은 예외 사항이 발생하면 에러 메시지를파이썬 셸 창에 출력

In [2]:
def getTourismStatsItem(yyyymm, national_code, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    
    parameters = "?_type=json&serviceKey=" + ServiceKey
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    
    url = service_url + parameters
    
    retData = getRequestUrl(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

- 02~09행 출입국관광통계서비스의 오픈 API 상세정보 페이지에서 찾은 서비스 URL, 요청매개변수 정보, 발급받은 인증키를 사용하여 데이터 요청 URL을 구성
- 11행 구성한 url로 getRequestUrl() 함수를 호출해서 받은 응답(utf-8로디코드됨)을 responseDecode에 저장
- 16행 서버에서 받은 JSON 형태의 응답 객체를 파이썬 객체로 로드하여 반환

In [3]:
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName = ''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12))
    isDataEnd = 0
    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            if(isDataEnd ==1): break
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd)
            if (jsonData['response']['header']['resultMsg'] == 'OK'):
                if jsonData['response']['body']['items'] == '':
                    isDataEnd =1
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))
                    break
                print(json.dumps(jsonData, indent = 4, sort_keys = True, ensure_ascii = False))
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('-----------------------------------------------------')
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])
    return (jsonResult, result, natName, ed, dataEND)

- 05행 수집할 데이터의 끝 날짜인 dataEND를 nEndYear의 12월로 설정한다.
- 06행 수집한 데이터의 끝인지 확인하기 위한 플래그인 isDataEnd를 0으로 설정한다.
- 09행 데이터 끝 플래그인 isdataEnd를 확인하여 플래그가 설정되어 있으면 작업을 중단한다.
- 10행 수집할 연도와 월을 여섯 자리로 맞추어 yyyymm에 저장
- 11행 getTourismStatsItem()을 호출해 받은 월 데이터를 jsonData에 저장
- 12행 응답 데이터가 정상인지 확인
- 13~17행 ['items'] 항목에 값이 없으면 출입국관광통계 데이터가 아직 들어가지 않은 마지막 월이므로 날짜를 dataEND에 저장하고 데이터 수집 작업을 중단
- 18행 수집한 월 데이터인 jsonData 내용을 확인할 수 있게 파이썬 셸 창에 출력
- 19~20행 수집한 국가 이름인 ['natKorNm'] 항목의 값에서 띄어쓰기를 제거하고 natName에 저장
- 21행 수집한 월의 데이터 수인 ['num'] 항목의 값을 num에 저장
- 22행 수집한 출입국 구분 데이터인 ['ed'] 항목의 값을 ed에 저장
- 25행 수집한 국가 이름(natName), 국가 코드(nat_cd), 날짜(yyyymm), 데이터 수(num)를 딕셔너리 자료형으로 구성하여 jsonResult 리스트에 원소로 추가
- 26행 수집한 국가 이름(natName), 국가 코드(nat_cd), 날짜(yyyymm), 데이터 수(num)를 result 리스트에 원소로 추가
- 27행수집하여 정리한 데이터를 반환

In [4]:
def main():
    jsonResult = []
    result = []
    
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"
    
    jsonResult, result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear)
    
    with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND),'w', encoding = 'utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True,ensure_ascii = False)
        
        outfile.write(jsonFile)
        
    columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear,dataEND), index=False, encoding='cp949')

- 06행 데이터를 수집할 국가 코드를 입력
- 07행 데이터를 수집할 시작 연도를 입력
- 08행 데이터를 수집할 마지막 연도를 입력
- 11행 getTourismStatsService() 함수를 호출하여 반환받은 수집 데이터를 jsonResult,result, natName, dataEND에 저장
- 13~16행 수집 데이터를 딕셔너리의 리스트로 저장한 jsonResult를 json.dumps()를 통해json 객체로 변환한 후 JSON 파일에 저장
- 18행 데이터프레임에 만들 컬럼명을 리스트로 만듬
- 19행 수집 데이터를 리스트로 저장한 result를 데이터프레임으로 변환
- 20행 데이터프레임 객체인 result_df를 CSV 파일로 저장

In [5]:
if __name__ == '__main__':
    main()

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : 130
데이터를 몇 년부터 수집할까요? : 2017
데이터를 몇 년까지 수집할까요? : 2020
[2022-01-01 20:03:43.089381] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일  본",
                    "num": 154862,
                    "rnum": 1,
                    "ym": 201701
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 일본_201701 : 154862 ]
-----------------------------------------------------
[2022-01-01 20:03:43.107249] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
            

[2022-01-01 20:03:43.458340] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일  본",
                    "num": 247847,
                    "rnum": 1,
                    "ym": 201809
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 일본_201809 : 247847 ]
-----------------------------------------------------
[2022-01-01 20:03:43.476341] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일  본",
                    "num": 290468,
 

[2022-01-01 20:03:43.833251] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일  본",
                    "num": 498,
                    "rnum": 1,
                    "ym": 202006
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 일본_202006 : 498 ]
-----------------------------------------------------
[2022-01-01 20:03:43.854251] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일  본",
                    "num": 755,
          

main()을 호출하여 시작